# Лабораторная работа 5

Рассматреть таблицу `beverage_r.csv` и решить три задачи:
1. Сделать разбиение 66/33 на обучающую и тестовую выборки
2. Разделить те же данные на четыре одинаковых по размеру части
3. Построить две стратифицированные подвыборки по признакам `COKE` ("кока-кола") и `SEVENUP` ("минералка")

Во всех пунктах фиксируем зерно генератора, чтобы результаты можно было воспроизвести

In [2]:
import pandas as pd
import numpy as np


DATA_PATH = "beverage_r.csv"
RANDOM_SEED = 11242025

df = pd.read_csv(DATA_PATH, sep=';', index_col='numb.obs')
shuffled = df.sample(frac=1, random_state=RANDOM_SEED)
print(f'Размерность набора: {df.shape}')
df.head()

Размерность набора: (34, 8)


,COKE,D_COKE,D_PEPSI,D_7UP,PEPSI,SPRITE,TAB,SEVENUP
numb.obs,,,,,,,,
1,1,0,0,0,1,1,0,1
2,1,0,0,0,1,0,0,0
3,1,0,0,0,1,0,0,0
4,0,1,0,1,0,0,1,0
5,1,0,0,0,1,0,0,0


## Задача 1

**Условие**

Разделить таблицу данных так, чтобы в обучающую выборку попало 66% наблюдений, а в тестовую выборку 33% наблюдений

**Решение**

In [3]:
def split_by_ratio(frame, train_ratio=0.66, seed=RANDOM_SEED):
    rng = np.random.default_rng(seed)
    idx = frame.index.to_numpy().copy()
    rng.shuffle(idx)
    train_size = int(round(len(frame) * train_ratio))
    train_idx = idx[:train_size]
    test_idx = idx[train_size:]
    return frame.loc[train_idx], frame.loc[test_idx]

train_df, test_df = split_by_ratio(df)
print(f'Обучающая выборка: {len(train_df)} наблюдений ({len(train_df)/len(df):.1%})')
print(f'Тестовая выборка:   {len(test_df)} наблюдений ({len(test_df)/len(df):.1%})')
train_df.head()

Обучающая выборка: 22 наблюдений (64.7%)
Тестовая выборка:   12 наблюдений (35.3%)


,COKE,D_COKE,D_PEPSI,D_7UP,PEPSI,SPRITE,TAB,SEVENUP
numb.obs,,,,,,,,
8,1,1,0,0,1,1,0,1
5,1,0,0,0,1,0,0,0
29,1,0,0,0,0,1,0,0
14,1,0,0,0,0,1,0,0
22,1,0,0,0,1,0,0,0


In [4]:
train_size = int(round(len(shuffled) * 0.66))
train_df = shuffled.iloc[:train_size]
test_df = shuffled.iloc[train_size:]

print(f"Train: {len(train_df)} наблюдений ({len(train_df)/len(df):.1%})")
print(f"Test:  {len(test_df)} наблюдений ({len(test_df)/len(df):.1%})")

Train: 22 наблюдений (64.7%)
Test:  12 наблюдений (35.3%)


## Задача 2

**Условие**

Разделить таблицу данных на четыре одинаковых части

**Решение**

In [5]:
shuffled = df.sample(frac=1, random_state=RANDOM_SEED)
folds = np.array_split(shuffled, 4)
for idx, fold in enumerate(folds, start=1):
    print(f'Часть {idx}: {len(fold)} наблюдений')
folds[0].head()

Часть 1: 9 наблюдений
Часть 2: 9 наблюдений
Часть 3: 8 наблюдений
Часть 4: 8 наблюдений


d:\Users\Legion\anaconda3\envs\py14_env\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


,COKE,D_COKE,D_PEPSI,D_7UP,PEPSI,SPRITE,TAB,SEVENUP
numb.obs,,,,,,,,
12,0,1,0,0,0,0,1,0
14,1,0,0,0,0,1,0,0
28,1,0,0,0,0,1,0,1
26,0,1,0,1,0,0,1,0
25,0,1,1,1,0,0,0,0


## Задача 3

**Условие**

Разделить таблицу данных на две стратифицированные подвыборки по переменным кока-кола и минералка.

**Решение**

In [6]:
STRAT_COLS = ['COKE', 'SEVENUP']

def stratified_two_subsamples(frame, strat_cols, seed=RANDOM_SEED):
    rng = np.random.default_rng(seed)

    part_a = []
    part_b = []

    # перемешивание страт и деление
    for combo, group in frame.groupby(strat_cols):
        idx = group.index.to_numpy().copy()
        rng.shuffle(idx)

        split_point = int(np.ceil(len(idx) / 2))
        part_a.append(frame.loc[idx[:split_point]])
        part_b.append(frame.loc[idx[split_point:]])

    # объединение
    part_a = pd.concat(part_a)
    part_b = pd.concat(part_b)

    # перемешивание после объединения
    part_a = part_a.sample(frac=1, random_state=seed)
    part_b = part_b.sample(frac=1, random_state=seed)

    return part_a, part_b

a_df, b_df = stratified_two_subsamples(df, STRAT_COLS)
print(f'Первая подвыборка: {len(a_df)} строк, вторая: {len(b_df)} строк')
print('\nСравнение долей по стратам:')
summary = (
    df.groupby(STRAT_COLS).size().to_frame('total')
    .join(a_df.groupby(STRAT_COLS).size().to_frame('part_a'), how='left')
    .join(b_df.groupby(STRAT_COLS).size().to_frame('part_b'), how='left')
)
summary.fillna(0, inplace=True)

summary['part_a_pct'] = summary['part_a'] / len(a_df)
summary['part_b_pct'] = summary['part_b'] / len(b_df)
summary

Первая подвыборка: 18 строк, вторая: 16 строк

Сравнение долей по стратам:


total  part_a  part_b  part_a_pct  part_b_pct
COKE SEVENUP                                               
0    0           13       7     6.0    0.388889       0.375
     1            1       1     0.0    0.055556       0.000
1    0           12       6     6.0    0.333333       0.375
     1            8       4     4.0    0.222222       0.250